In [47]:
import pandas as pd
import arxiv
import os
import networkx as nx
import itertools

In [5]:
client=arxiv.Client()

In [6]:
search = arxiv.Search(
  query = "misinformation",
  max_results = 10000,
  sort_by = arxiv.SortCriterion.SubmittedDate
)

In [7]:
results=client.results(search)

In [8]:
all_results = list(results)

In [9]:
all_results[0]

arxiv.Result(entry_id='http://arxiv.org/abs/2410.18866v1', updated=datetime.datetime(2024, 10, 24, 15, 51, 4, tzinfo=datetime.timezone.utc), published=datetime.datetime(2024, 10, 24, 15, 51, 4, tzinfo=datetime.timezone.utc), title='The Cat and Mouse Game: The Ongoing Arms Race Between Diffusion Models and Detection Methods', authors=[arxiv.Result.Author('Linda Laurier'), arxiv.Result.Author('Ave Giulietta'), arxiv.Result.Author('Arlo Octavia'), arxiv.Result.Author('Meade Cleti')], summary='The emergence of diffusion models has transformed synthetic media generation,\noffering unmatched realism and control over content creation. These\nadvancements have driven innovation across fields such as art, design, and\nscientific visualization. However, they also introduce significant ethical and\nsocietal challenges, particularly through the creation of hyper-realistic\nimages that can facilitate deepfakes, misinformation, and unauthorized\nreproduction of copyrighted material. In response, the

In [42]:
authorship=[]
for authors in all_results:
    author_list=[]
    for i in authors.authors:
        author_list.append(i.name)
    authorship.append(author_list)

In [48]:
authorship[0]

['Linda Laurier', 'Ave Giulietta', 'Arlo Octavia', 'Meade Cleti']

In [49]:
G=nx.Graph()

In [51]:
edges=[]
for val in authorship:
    edges.extend(list(itertools.combinations(val, 2)))

In [55]:
G.add_edges_from(edges)

In [56]:
len(list(G.nodes))

4777

In [57]:
sorted(nx.degree_centrality(G).items(), key=lambda x:x[1], reverse=True)[:5]

[('Preslav Nakov', 0.020309882747068676),
 ('Iason Gabriel', 0.014447236180904523),
 ('Lisa Anne Hendricks', 0.014447236180904523),
 ('Sasha Brown', 0.014447236180904523),
 ('Conor Griffin', 0.014447236180904523)]